In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import fa_support
import evall
import random

In [2]:
train =  np.load("new_fa_train_data.npy")
#test = np.load("new_fa_test_data.npy")


In [3]:
numbers = [  308, 8, 53, 3, 13, 5, 3, 78]
names = ['brand_id',
         'category', 'colour', 'divisioncode', 'itemcategorycode',
         'itemfamilycode', 'itemseason', 'productgroup']
dic = dict(zip(names,numbers))

In [4]:
user_emb_dim = sum(dic.values())

D_emb_dim = 50

G_emb_dim = 50

hidden_dim = 128
alpha = 0

# Initializer
init = tf.initializers.glorot_normal()

'''Generator and Discriminator Attribute Embeddings'''
#D_price_embs = tf.keras.layers.Embedding(input_dim = dic['pricetype'], output_dim = D_emb_dim,
 #                                         trainable=True, weights = [init(shape=( dic['pricetype'],D_emb_dim))])
D_brand_embs = tf.keras.layers.Embedding(input_dim = dic['brand_id'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['brand_id'],D_emb_dim))])
D_category_embs = tf.keras.layers.Embedding(input_dim = dic['category'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['category'],D_emb_dim))])
D_colour_embs = tf.keras.layers.Embedding(input_dim = dic['colour'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['colour'],D_emb_dim))])
D_div_embs = tf.keras.layers.Embedding(input_dim = dic['divisioncode'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['divisioncode'],D_emb_dim))])
D_itemcat_embs = tf.keras.layers.Embedding(input_dim = dic['itemcategorycode'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemcategorycode'],D_emb_dim))])
D_itemfam_embs = tf.keras.layers.Embedding(input_dim = dic['itemfamilycode'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemfamilycode'],D_emb_dim))])
D_season_embs = tf.keras.layers.Embedding(input_dim = dic['itemseason'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemseason'],D_emb_dim))])
D_prod_embs = tf.keras.layers.Embedding(input_dim = dic['productgroup'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['productgroup'],D_emb_dim))])


#G_price_embs = tf.keras.layers.Embedding(input_dim = dic['pricetype'], output_dim = G_emb_dim,
#                                               trainable=True, weights = [init(shape=( dic['pricetype'],G_emb_dim))])
G_brand_embs = tf.keras.layers.Embedding(input_dim = dic['brand_id'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['brand_id'],G_emb_dim))])
G_category_embs = tf.keras.layers.Embedding(input_dim = dic['category'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['category'],G_emb_dim))])
G_colour_embs = tf.keras.layers.Embedding(input_dim = dic['colour'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['colour'],G_emb_dim))])
G_div_embs = tf.keras.layers.Embedding(input_dim = dic['divisioncode'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['divisioncode'],G_emb_dim))])
G_itemcat_embs = tf.keras.layers.Embedding(input_dim = dic['itemcategorycode'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemcategorycode'],G_emb_dim))])
G_itemfam_embs = tf.keras.layers.Embedding(input_dim = dic['itemfamilycode'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemfamilycode'],G_emb_dim))])
G_season_embs = tf.keras.layers.Embedding(input_dim = dic['itemseason'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['itemseason'],G_emb_dim))])
G_prod_embs = tf.keras.layers.Embedding(input_dim = dic['productgroup'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['productgroup'],G_emb_dim))])


# Model input sizes
G_input_size =  G_emb_dim*8
D_input_size = user_emb_dim + D_emb_dim*8


In [5]:
def generator_input( brand_id, category, colour, divisioncode, itemcategorycode, itemfamilycode, itemseason, productgroup):
    emb_dic = {}
#    dic["pricetype"] = G_brand_embs(tf.constant(pricetype))
    dic["brand_id"] = G_brand_embs(tf.constant(brand_id))
    dic["category"] = G_brand_embs(tf.constant(category))
    dic["colour"] = G_brand_embs(tf.constant(colour))
    dic["divisioncode"] = G_brand_embs(tf.constant(divisioncode))
    dic["itemcategorycode"] = G_brand_embs(tf.constant(itemcategorycode))
    dic["itemfamilycode"] = G_brand_embs(tf.constant(itemfamilycode))
    dic["itemseason"] = G_brand_embs(tf.constant(itemseason))
    dic["productgroup"] = G_brand_embs(tf.constant(productgroup))
    emb = tf.keras.layers.concatenate(list(dic.values()), 1)
    return emb

# Generates user based on concatenation of all attributes
def generator():
    bc_input = tf.keras.layers.Input(shape=(G_input_size))
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', activity_regularizer = 'l2')(bc_input)
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', activity_regularizer = 'l2')(x)
    x = tf.keras.layers.Dense(user_emb_dim, activation ='sigmoid', activity_regularizer = 'l2')(x)
    g_model = tf.keras.models.Model(bc_input, x, name = 'generator')
    return g_model
g_model = generator()

def discriminator_input( brand_id, category, colour, divisioncode, itemcategorycode, itemfamilycode, 
                        itemseason, productgroup, user_emb):
    emb_dic = {}
#    dic["pricetype"]  = D_brand_embs(tf.constant(pricetype))
    dic["brand_id"]  = D_brand_embs(tf.constant(brand_id))
    dic["category"]  = D_brand_embs(tf.constant(category))
    dic["colour"]  = D_brand_embs(tf.constant(colour))
    dic["divisioncode"]  = D_brand_embs(tf.constant(divisioncode))
    dic["itemcategorycode"]  = D_brand_embs(tf.constant(itemcategorycode))
    dic["itemfamilycode"]  = D_brand_embs(tf.constant(itemfamilycode))
    dic["itemseason"]  = D_brand_embs(tf.constant(itemseason))
    dic["productgroup"]  = D_brand_embs(tf.constant(productgroup))
    user_emb = tf.cast(user_emb, dtype=float)
    emb = tf.keras.layers.concatenate(list(dic.values()), 1)
    final_emb = tf.keras.layers.concatenate([emb, user_emb], 1)
    return final_emb

def discriminator():
    d_input = tf.keras.layers.Input(shape=(D_input_size))
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', kernel_regularizer = 'l2')(d_input)
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', kernel_regularizer = 'l2')(x)
    x = tf.keras.layers.Dense(1)(x)
    model = tf.keras.models.Model(d_input, x, name = 'discriminator')
    return model
d_model = discriminator()

In [6]:
ua_matrix = np.load("new_fa_ua_matrix.npy")
ia_matrix = np.load("new_fa_ia_matrix.npy")

In [7]:
'''Loss functions'''
def generator_loss(fake_user):
    return -tf.reduce_mean(fake_user)

def discriminator_loss(real, fake):
    logit = tf.reduce_mean(fake- real)
    return logit

def counter_loss(counter):
    return tf.reduce_mean(counter)

def discriminator_counter_loss(real, fake, counter):
    logit = tf.reduce_mean(real - counter - fake)
    return logit
''' Cross entropy loss means
def generator_loss(d_logits):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits, labels=tf.ones_like(d_logits)))

def discriminator_loss(real, fake):
    logit = tf.reduce_mean(fake-real)
    r = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real, labels=tf.ones_like(real)))
    f = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake, labels=tf.zeros_like(fake)))
    return r+f

def counter_loss(counter):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=counter, labels=tf.zeros_like(counter))) 
'''

' Cross entropy loss means\ndef generator_loss(d_logits):\n    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits, labels=tf.ones_like(d_logits)))\n\ndef discriminator_loss(real, fake):\n    logit = tf.reduce_mean(fake-real)\n    r = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real, labels=tf.ones_like(real)))\n    f = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake, labels=tf.zeros_like(fake)))\n    return r+f\n\ndef counter_loss(counter):\n    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=counter, labels=tf.zeros_like(counter))) \n'

In [8]:
# WGAN Class
class WGAN(tf.keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        discriminator_extra_steps=5,
        batch_size = 1000
    ):
        super(WGAN, self).__init__()
        self.discriminator = d_model
        self.generator = g_model
        self.d_steps = discriminator_extra_steps
        self.batch_size = batch_size
        self.k = 10        
        self.index = 0 
        self.c_index = 0 
        self.gp_weight = 10
        self.eval_steps = 0
        self.max_p10 = .3 
        self.max_g10 = .3
        self.max_m10 = .3
        self.max_p20 = .3
        self.max_g20 = .3
        self.max_m20 = .3
    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn,c_loss_fn, run_eagerly):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.c_loss_fn = c_loss_fn
        self.g_loss_fn = g_loss_fn
        self.run_eagerly = run_eagerly



    def gradient_penalty(self, batch_size, real_users, fake_users,  brand_id,\
                                    category, colour, divisioncode, itemcategorycode, itemfamilycode, \
                                    itemseason, productgroup):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size,1], 0.0, 1.0)
        diff = fake_users - real_users
        interpolated = real_users + alpha * diff

        with tf.GradientTape() as tape:
            tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            interpolated_input = discriminator_input(   brand_id,\
                                    category, colour, divisioncode, itemcategorycode, itemfamilycode, \
                                    itemseason, productgroup, interpolated)
            pred = self.discriminator(interpolated_input)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = tape.gradient(pred, [interpolated])[0] #+1e-10
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=1))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_users):
        self.eval_steps +=1 
        c_batch_size = 2*self.batch_size
        for i in range(self.d_steps):

            with tf.GradientTape() as tape:
                # Get batch data
                country, postcode, loyal, gender,  brand_id, category, colour, divisioncode, \
                itemcategorycode, itemfamilycode, itemseason, productgroup, real_users, items = fa_support.get_batchdata(self.index, self.index + self.batch_size)
                # Get batch of counter examples
                '''counter_brand_id, counter_category, counter_colour, counter_divisioncode, \
                counter_itemcategorycode, counter_itemfamilycode, counter_itemseason, \
                counter_productgroup,  counter_users = fa_support.get_counter_batch(self.c_index, self.c_index + c_batch_size)'''
                # Generate fake users from attributes
                g_input0 = generator_input(  brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup)
                fake_users = self.generator(g_input0)
                # Get the logits for the fake users
                d_input0 = discriminator_input( brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup, fake_users)
                fake_logits = self.discriminator(d_input0)
                # Get the logits for the real user
                d_input1 = discriminator_input(  brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup, real_users)
                real_logits = self.discriminator(d_input1)
                # Get logits for counter examples
                
                '''d_input2 = discriminator_input( counter_brand_id, counter_category, counter_colour, counter_divisioncode, \
                counter_itemcategorycode, counter_itemfamilycode, counter_itemseason, \
                counter_productgroup,  counter_users)
                counter_logits = self.discriminator(d_input2)'''
                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_logits, fake_logits)
                #c_loss = self.c_loss_fn(counter_logits)
                # Get gradient penalty
                gp = self.gradient_penalty(self.batch_size, real_users, fake_users, brand_id,\
                                    category, colour, divisioncode, itemcategorycode, itemfamilycode, \
                                    itemseason, productgroup)
                # Later add counter loss
                d_loss = d_cost + gp*self.gp_weight # + c_loss

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        # Train the generator
        with tf.GradientTape() as tape:

            # Generate fake images using the generator
            g_input1 = generator_input(  brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup)
            gen_users = self.generator(g_input1)
            # Get the discriminator logits for fake images
            d_input2 = discriminator_input(  brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup, gen_users)
            gen_logits = self.discriminator(d_input2)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        if self.eval_steps %760==0:
            p_at_10,G_at_10,M_at_10 = wgan.test_step(10)
            p_at_20,G_at_20,M_at_20 = wgan.test_step(20)
            if p_at_10 > self.max_p10:
                self.max_p10 = p_at_10
            if G_at_10 > self.max_g10:
                self.max_g10 = G_at_10
            if M_at_10 > self.max_m10:
                self.max_m10 = M_at_10
            if p_at_20 > self.max_p20:
                self.max_p20 = p_at_20
            if G_at_20 > self.max_g20:
                self.max_g20 = G_at_20
            if M_at_20 > self.max_m20:
                self.max_m20 = M_at_20
            
            return {"d_loss": d_loss, "g_loss": g_loss, "p10":p_at_10,
                           "G10":G_at_10,"M10":M_at_10, "p20": p_at_20,"G20":G_at_20,"M20":M_at_20}
        else:
            return {"d_loss": d_loss, "g_loss": g_loss}

    def test_step(self, k):
        country, postcode, loyal, gender,  brand_id, category, colour, divisioncode, \
        itemcategorycode, itemfamilycode, itemseason, productgroup, item = fa_support.get_testdata(0,5000)
        
        test_BATCH_SIZE = country.size
        g_input1 = generator_input( brand_id, category, colour, divisioncode, \
                                           itemcategorycode, itemfamilycode, itemseason, productgroup)
        gen_users = self.generator(g_input1)
        sim_users = fa_support.get_intersection_similar_user(gen_users, k)
        count = 0
        for i, userlist in zip(item, sim_users):       
            for u in userlist:
                if np.sum(ia_matrix[i] + ua_matrix[u] == 2) ==8:
                    count = count + 1            
        p_at_10 = round(count/(test_BATCH_SIZE * k), 4)

        ans = 0.0
        RS = []
        for i, userlist in zip(item, sim_users):  
            r=[]
            for u in userlist:
                if np.sum(ia_matrix[i] + ua_matrix[u] == 2) ==8:
                    r.append(1)
                else:
                    r.append(0)
            RS.append(r)
            ans = ans + evall.ndcg_at_k(r, k, method=1)
        G_at_10 = ans/test_BATCH_SIZE
        M_at_10 = evall.mean_average_precision(RS)

        return p_at_10,G_at_10,M_at_10

In [9]:
# Fit 
epochs = 50
# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=discriminator,
    generator=generator,
    discriminator_extra_steps=3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
    c_loss_fn = counter_loss,
    run_eagerly=True)

# Start training the model.
fit = wgan.fit(train, batch_size=100, epochs=epochs, verbose=True)
print(wgan.test_step(10), "\n", wgan.test_step(20))

Epoch 1/50
762/762 [==============================] - 282s 371ms/step - d_loss: -2.0680 - g_loss: -3.4180 - p10: 0.0635 - G10: 0.1632 - M10: 0.1312 - p20: 0.0490 - G20: 0.1800 - M20: 0.1286
Epoch 2/50
762/762 [==============================] - 285s 374ms/step - d_loss: -1.5877 - g_loss: -9.3851 - p10: 0.1317 - G10: 0.2494 - M10: 0.2040 - p20: 0.1217 - G20: 0.2682 - M20: 0.1983
Epoch 3/50
762/762 [==============================] - 285s 375ms/step - d_loss: -1.2013 - g_loss: -10.0814 - p10: 0.2098 - G10: 0.3348 - M10: 0.2874 - p20: 0.2021 - G20: 0.3577 - M20: 0.2803
Epoch 4/50
762/762 [==============================] - 286s 375ms/step - d_loss: -0.9304 - g_loss: -10.8972 - p10: 0.3151 - G10: 0.4431 - M10: 0.3898 - p20: 0.3064 - G20: 0.4646 - M20: 0.3820
Epoch 5/50
762/762 [==============================] - 286s 376ms/step - d_loss: -0.7007 - g_loss: -12.3077 - p10: 0.4187 - G10: 0.5313 - M10: 0.4872 - p20: 0.4016 - G20: 0.5515 - M20: 0.4825
Epoch 6/50
762/762 [===========================

MemoryError: Unable to allocate 3.22 GiB for an array with shape (5000, 86391) and data type float64

In [19]:
wgan.max_p20

0.3